In [19]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import itertools

import random

import pandas as pd

from sklearn.metrics import roc_curve, auc

ignore_colms = ['sender', 'receiver', 'transaction']
label_colm = 'transaction'

In [25]:
def load_features_file(convert_to_bin_trans, svd_k, nmf_n, unq_senders, unq_receivers):
    filename_postfix = '_bin{}_svd{}_nmf{}_snd{}_rcv{}.csv'.format(str(convert_to_bin_trans), str(svd_k), str(nmf_n), str(unq_senders), str(unq_receivers))
    ftrain = pd.read_csv('features/train' + filename_postfix, index_col=0)
    ftest = pd.read_csv('features/test' + filename_postfix, index_col=0)
    return ftrain, ftest


def get_train_test_matricies(traindf, testdf):
    train_X = traindf.drop(ignore_colms, axis=1).as_matrix()
    train_Y = traindf[label_colm].values
    
    test_X = testdf.drop(ignore_colms, axis=1).as_matrix()
    test_true_Y = testdf[label_colm].values
    return train_X, train_Y, test_X, test_true_Y

In [26]:
# Load DFs
bin_trans = True
svd_k = 50
nmf_n = 12
unq_senders = 250
unq_receivers = 100

traindf, testdf = load_features_file(bin_trans, svd_k, nmf_n, unq_senders, unq_receivers)
train_X, train_Y, test_X, test_true_Y = get_train_test_matricies(traindf, testdf)

In [33]:
from itertools import cycle

from time import time
from operator import itemgetter

from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegressionCV, Perceptron

from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import  cross_val_score

from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from scipy import interp
from sklearn import svm
from sklearn.metrics import roc_curve, precision_recall_curve, auc, accuracy_score, classification_report, confusion_matrix, average_precision_score
from sklearn.naive_bayes import MultinomialNB

import time

In [32]:
# Function to run and get stats for any classifier
def simple_any(clf, train_features, train_labels, test_features, test_true_labels):
    start_time = time.time()
    stats = dict()
    clf = clf.fit(train_features, train_labels)

    train_predicted_labels = clf.predict(train_features)
    test_predicted_labels = clf.predict(test_features)

    y_score = clf.predict_proba(test_features)
    
    stats['time'] = time.time() - start_time
    stats['false_pv_arr'], stats['true_pv_arr'], thresholds = roc_curve(test_true_labels, y_score[:, 1])
    stats['precision_arr'], stats['recall_arr'], prthres = precision_recall_curve(test_true_labels, y_score[:, 1])
    stats['auc'] = auc(stats['false_pv_arr'], stats['true_pv_arr'])
    stats['aver_pr'] = average_precision_score(test_true_labels, y_score[:, 1])
    stats['precision_recall_f1'] = classification_report(test_true_labels, test_predicted_labels)
    stats['confusion_matrix'] = confusion_matrix(test_true_labels, test_predicted_labels)
    stats['test_accuracy'] = accuracy_score(test_true_labels, test_predicted_labels)
    stats['train_accuracy'] = accuracy_score(train_labels, train_predicted_labels)
    return stats